# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [34]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [35]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [36]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [37]:
all_features = ['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 
                'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
                'sqft_basement', 'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [38]:
model_all = graphlab.linear_regression.create(
    sales, target = 'price', features = all_features, validation_set = None, l2_penalty = 0., l1_penalty = 1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.733777     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.789395     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [39]:
model_all.get('coefficients').print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [40]:
(training_and_validation, testing) = sales.random_split(0.9, seed = 1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed = 1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [41]:
best_lambda = 1.0
best_rss = 9.99e20

for penalty in np.logspace(1, 7, num = 13):
    mod = graphlab.linear_regression.create(training, 
                                            target = 'price', 
                                            features = all_features, 
                                            validation_set = None, 
                                            l2_penalty = 0., 
                                            l1_penalty = penalty,
                                            verbose = False)
    preds = mod.predict(validation)
    error = preds - validation['price']
    rss = sum(error ** 2)
    if rss < best_rss:
        best_rss = rss
        best_lambda = penalty
    print 'rss:', rss, 'lambda:', penalty 

print ''
print 'best lambda:', best_lambda, '(rss:', best_rss, ')'

rss: 6.25766285142e+14 lambda: 10.0
rss: 6.25766285362e+14 lambda: 31.6227766017
rss: 6.25766286058e+14 lambda: 100.0
rss: 6.25766288257e+14 lambda: 316.227766017
rss: 6.25766295212e+14 lambda: 1000.0
rss: 6.25766317206e+14 lambda: 3162.27766017
rss: 6.25766386761e+14 lambda: 10000.0
rss: 6.25766606749e+14 lambda: 31622.7766017
rss: 6.25767302792e+14 lambda: 100000.0
rss: 6.25769507644e+14 lambda: 316227.766017
rss: 6.25776517727e+14 lambda: 1000000.0
rss: 6.25799062845e+14 lambda: 3162277.66017
rss: 6.25883719085e+14 lambda: 10000000.0

best lambda: 10.0 (rss: 6.25766285142e+14 )


In [42]:
best_lambda = 10.0

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [43]:
best_mod = graphlab.linear_regression.create(training, 
                                             target = 'price', 
                                             features = all_features, 
                                             validation_set = None, 
                                             l2_penalty = 0., 
                                             l1_penalty = penalty,
                                             verbose = False)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [44]:
coef_table = best_mod.get('coefficients')
print coef_table[['name', 'value']].print_rows(num_rows = 18)
best_mod['coefficients']['value'].nnz()

+------------------+--------------------+
|       name       |       value        |
+------------------+--------------------+
|   (intercept)    |   19604.7163508    |
|     bedrooms     |   7922.62023075    |
| bedrooms_square  |   928.112147889    |
|    bathrooms     |    25392.743443    |
|   sqft_living    |   39.1083048767    |
| sqft_living_sqrt |   1123.95233925    |
|     sqft_lot     | -0.000823987151992 |
|  sqft_lot_sqrt   |   147.859054391    |
|      floors      |    21154.353574    |
|  floors_square   |    12871.042877    |
|    waterfront    |   598538.378522    |
|       view       |   93089.3048849    |
|    condition     |   6584.64358167    |
|      grade       |   6200.37319739    |
|    sqft_above    |   43.2627983054    |
|  sqft_basement   |   122.027231745    |
|     yr_built     |   9.39509533202    |
|   yr_renovated   |   55.3271344922    |
+------------------+--------------------+
[18 rows x 2 columns]

None


18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

# CONTINUE HERE

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [20]:
l1_penalty_values = np.logspace(8, 10, num = 20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [21]:
for penalty in l1_penalty_values:
    mod = graphlab.linear_regression.create(training, 
                                            target = 'price', 
                                            features = all_features, 
                                            validation_set = None, 
                                            l2_penalty = 0., 
                                            l1_penalty = penalty,
                                            verbose = False)
    mod_coeffs = mod['coefficients']['value']
    print 'no. coeffs:', mod_coeffs.nnz(), 'lambda:', penalty


no. coeffs: 18 lambda: 100000000.0
no. coeffs: 18 lambda: 127427498.57
no. coeffs: 18 lambda: 162377673.919
no. coeffs: 18 lambda: 206913808.111
no. coeffs: 17 lambda: 263665089.873
no. coeffs: 17 lambda: 335981828.628
no. coeffs: 17 lambda: 428133239.872
no. coeffs: 17 lambda: 545559478.117
no. coeffs: 17 lambda: 695192796.178
no. coeffs: 16 lambda: 885866790.41
no. coeffs: 15 lambda: 1128837891.68
no. coeffs: 15 lambda: 1438449888.29
no. coeffs: 13 lambda: 1832980710.83
no. coeffs: 12 lambda: 2335721469.09
no. coeffs: 10 lambda: 2976351441.63
no. coeffs: 6 lambda: 3792690190.73
no. coeffs: 5 lambda: 4832930238.57
no. coeffs: 3 lambda: 6158482110.66
no. coeffs: 1 lambda: 7847599703.51
no. coeffs: 1 lambda: 10000000000.0


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [22]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [23]:
l1_penalty_values = np.linspace(l1_penalty_min, l1_penalty_max, 20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [27]:
for l1_penalty in l1_penalty_values:
    mod = graphlab.linear_regression.create(training, 
                                            target = 'price', 
                                            features = all_features, 
                                            validation_set = None, 
                                            l2_penalty = 0., 
                                            l1_penalty = l1_penalty,
                                            verbose = False)
    mod_coeffs = mod['coefficients']['value']
    num_params = mod_coeffs.nnz()
    preds = mod.predict(validation)
    error = preds - validation['price']
    rss = sum(error ** 2)
    print 'rss:', rss, 'no. params:', num_params, 'lambda:', l1_penalty
    

rss: 9.66925692362e+14 no. params: 10 lambda: 2976351441.63
rss: 9.74019450085e+14 no. params: 10 lambda: 3019316638.95
rss: 9.81188367942e+14 no. params: 10 lambda: 3062281836.27
rss: 9.89328342459e+14 no. params: 10 lambda: 3105247033.59
rss: 9.98783211266e+14 no. params: 10 lambda: 3148212230.91
rss: 1.00847716702e+15 no. params: 10 lambda: 3191177428.24
rss: 1.01829878055e+15 no. params: 10 lambda: 3234142625.56
rss: 1.02824799221e+15 no. params: 10 lambda: 3277107822.88
rss: 1.03461690923e+15 no. params: 8 lambda: 3320073020.2
rss: 1.03855473594e+15 no. params: 8 lambda: 3363038217.52
rss: 1.04323723787e+15 no. params: 8 lambda: 3406003414.84
rss: 1.04693748875e+15 no. params: 7 lambda: 3448968612.16
rss: 1.05114762561e+15 no. params: 7 lambda: 3491933809.48
rss: 1.05599273534e+15 no. params: 7 lambda: 3534899006.8
rss: 1.06079953176e+15 no. params: 7 lambda: 3577864204.12
rss: 1.0657076895e+15 no. params: 6 lambda: 3620829401.45
rss: 1.06946433543e+15 no. params: 6 lambda: 366379

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [30]:
best_lambda7 = 3448968612.16
best_mod = graphlab.linear_regression.create(training, 
                                             target = 'price', 
                                             features = all_features, 
                                             validation_set = None, 
                                             l2_penalty = 0., 
                                             l1_penalty = best_lambda7,
                                             verbose = False)
best_mod['coefficients'].print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |